In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.preprocessing import StandardScaler
import umap
import hdbscan
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load extracted features
print("Loading extracted features...")
features_file = "data/extracted_features.csv"
df = pd.read_csv(features_file)

# Convert Time to datetime
df['Time'] = pd.to_datetime(df['Time'])

print(f"Loaded {df.shape[0]} rows with {df.shape[1]} features")

Loading extracted features...
Loaded 1526373 rows with 75 features


In [3]:
# Group features by category
liquidity_features = [col for col in df.columns if any(x in col for x in ['bid_ask_spread', 'imbalance', 'depth', 'slope'])]
volatility_features = [col for col in df.columns if any(x in col for x in ['volatility', 'zscore'])]
trend_features = [col for col in df.columns if any(x in col for x in ['return', 'trend', 'rsi'])]
volume_features = [col for col in df.columns if any(x in col for x in ['volume', 'trade', 'vwap'])]

# Print feature counts by category
print(f"Liquidity features: {len(liquidity_features)}")
print(f"Volatility features: {len(volatility_features)}")
print(f"Trend features: {len(trend_features)}")
print(f"Volume features: {len(volume_features)}")

Liquidity features: 37
Volatility features: 8
Trend features: 12
Volume features: 17


In [ ]:
# Feature selection - use the most important features from each category
selected_features = [
    # Liquidity
    'bid_ask_spread_bps', 'imbalance_lvl1', 'cum_depth_imbalance', 
    'bid_slope', 'ask_slope', 'mean_bid_price_spacing', 'mean_ask_price_spacing',
    'bid_depth_5lvl', 'ask_depth_5lvl',
    
    # Volatility
    'volatility_10s', 'volatility_30s', 'volatility_60s',
    'zscore_10s', 'zscore_30s', 'zscore_60s',
    
    # Trend
    'return_10s', 'return_30s', 'return_60s',
    'rsi_10s', 'rsi_30s', 'rsi_60s',
    'trend_slope_10s', 'trend_slope_30s', 'trend_slope_60s',
    
    # Volume
    'volume_10s', 'volume_30s', 'volume_60s',
    'volume_imbalance_10s', 'volume_imbalance_30s', 'volume_imbalance_60s',
    'avg_trade_size_30s'
]

# Add mid_price for visualization
if 'mid_price' in df.columns:
    price_col = 'mid_price'
else:
    # If mid_price is not available, calculate it
    if 'BidPriceL1' in df.columns and 'AskPriceL1' in df.columns:
        df['mid_price'] = (df['BidPriceL1'] + df['AskPriceL1']) / 2
        price_col = 'mid_price'
    else:
        price_col = None

# Filter out features that might not exist in the dataframe
existing_features = [f for f in selected_features if f in df.columns]
print(f"Using {len(existing_features)} features for clustering")

# Drop rows with NaN values in selected features
data_for_clustering = df[['Time'] + ([price_col] if price_col else []) + existing_features].dropna()
print(f"After dropping NaN values: {data_for_clustering.shape[0]} rows")

# Separate features and time
X = data_for_clustering[existing_features]
times = data_for_clustering['Time']

# Apply PCA for dimensionality reduction
print("Applying PCA...")
pca = PCA(n_components=0.95)  # Keep enough components to explain 95% of variance
X_pca = pca.fit_transform(X)
print(f"PCA reduced dimensions from {X.shape[1]} to {X_pca.shape[1]}")
print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Cumulative explained variance: {np.sum(pca.explained_variance_ratio_)}")

# Plot PCA components
plt.figure(figsize=(10, 6))
plt.bar(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_)
plt.xlabel('PCA Component')
plt.ylabel('Explained Variance Ratio')
plt.title('PCA Components Explained Variance')
plt.grid(True, alpha=0.3)
plt.savefig('final/pca_components.png', dpi=300, bbox_inches='tight')
plt.close()

# ----- KMEANS CLUSTERING WITH K=3 -----
print("\n----- K-MEANS CLUSTERING WITH K=3 -----")
# Apply K-means with fixed k=3
k = 3
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
kmeans_labels = kmeans.fit_predict(X_pca)

# ----- REGIME LABELING AND ANALYSIS -----
print("\n----- REGIME LABELING AND ANALYSIS -----")
def analyze_cluster_regimes(data, cluster_column, feature_columns):
    """Analyze and label regimes based on cluster characteristics"""
    regimes = {}
    
    # Get unique clusters, excluding noise (-1) if present
    unique_clusters = sorted([c for c in data[cluster_column].unique() if c != -1])
    
    for cluster in unique_clusters:
        cluster_data = data[data[cluster_column] == cluster]
        
        # Analyze trend characteristics
        trend_cols = [col for col in feature_columns if any(x in col for x in ['return', 'trend', 'rsi'])]
        trend_features = cluster_data[trend_cols].mean()
        
        # Simple trend detection
        trend_slope = trend_features['trend_slope_30s'] if 'trend_slope_30s' in trend_features else 0
        rsi = trend_features['rsi_30s'] if 'rsi_30s' in trend_features else 50
        
        if abs(trend_slope) < 0.0001:
            trend_regime = "Mean-Reverting"
        elif trend_slope > 0:
            trend_regime = "Upward-Trending"
        else:
            trend_regime = "Downward-Trending"
        
        # Analyze volatility characteristics
        vol_cols = [col for col in feature_columns if 'volatility' in col]
        volatility = cluster_data[vol_cols].mean().mean()
        
        if volatility > 0.0001:
            volatility_regime = "Volatile"
        else:
            volatility_regime = "Stable"
        
        # Analyze liquidity characteristics
        liquidity_cols = [col for col in feature_columns if any(x in col for x in ['spread', 'depth'])]
        liquidity_features = cluster_data[liquidity_cols].mean()
        
        spread_bps = liquidity_features['bid_ask_spread_bps'] if 'bid_ask_spread_bps' in liquidity_features else 0
        depth = (liquidity_features['bid_depth_5lvl'] + liquidity_features['ask_depth_5lvl']) / 2 if 'bid_depth_5lvl' in liquidity_features and 'ask_depth_5lvl' in liquidity_features else 0
        
        if spread_bps > 1.0 or depth < 10:
            liquidity_regime = "Illiquid"
        else:
            liquidity_regime = "Liquid"
        
        # Combine regimes
        combined_regime = f"{trend_regime} & {volatility_regime} & {liquidity_regime}"
        
        # Store regime analysis
        regimes[cluster] = {
            'name': combined_regime,
            'count': len(cluster_data),
            'percentage': len(cluster_data) / len(data) * 100,
            'trend_slope': trend_slope,
            'rsi': rsi,
            'volatility': volatility,
            'spread_bps': spread_bps,
            'depth': depth,
            'trend_regime': trend_regime,
            'volatility_regime': volatility_regime,
            'liquidity_regime': liquidity_regime
        }
    
    return regimes

# Add the cluster labels to the dataframe
data_for_clustering['kmeans_cluster'] = kmeans_labels

# Analyze regimes for K-means clustering
kmeans_regimes = analyze_cluster_regimes(data_for_clustering, 'kmeans_cluster', existing_features)

# Add regime labels to dataframe
data_for_clustering['kmeans_regime'] = data_for_clustering['kmeans_cluster'].map(
    {k: v['name'] for k, v in kmeans_regimes.items()}
)

# Print regime analysis
print("\nK-Means Regime Analysis:")
for cluster, info in kmeans_regimes.items():
    print(f"\nCluster {cluster} - {info['name']}")
    print(f"  Samples: {info['count']} ({info['percentage']:.2f}%)")
    print(f"  Trend: {info['trend_regime']} (Slope={info['trend_slope']:.6f}, RSI={info['rsi']:.2f})")
    print(f"  Volatility: {info['volatility_regime']} ({info['volatility']:.6f})")
    print(f"  Liquidity: {info['liquidity_regime']} (Spread={info['spread_bps']:.2f} bps, Depth={info['depth']:.2f})")

# ----- DIMENSIONALITY REDUCTION FOR VISUALIZATION -----
print("\n----- DIMENSIONALITY REDUCTION FOR VISUALIZATION -----")
# Apply t-SNE
print("Computing t-SNE projection...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
X_tsne = tsne.fit_transform(X_pca)

# Apply UMAP
print("Computing UMAP projection...")
reducer = umap.UMAP(random_state=42)
X_umap = reducer.fit_transform(X_pca)

# ----- VISUALIZATIONS -----
print("\n----- CREATING VISUALIZATIONS -----")

# Create a color palette for better visualizations
colors = sns.color_palette("tab10", k)

# 1. t-SNE visualization of K-means clusters
plt.figure(figsize=(12, 10))
for i in range(k):
    mask = kmeans_labels == i
    plt.scatter(X_tsne[mask, 0], X_tsne[mask, 1], c=[colors[i]], label=f"Cluster {i}: {kmeans_regimes[i]['name']}", alpha=0.7)
plt.title('t-SNE Visualization of Market Regimes (K-means)')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('final/tsne_kmeans_clusters.png', dpi=300, bbox_inches='tight')
plt.close()

# 2. UMAP visualization of K-means clusters
plt.figure(figsize=(12, 10))
for i in range(k):
    mask = kmeans_labels == i
    plt.scatter(X_umap[mask, 0], X_umap[mask, 1], c=[colors[i]], label=f"Cluster {i}: {kmeans_regimes[i]['name']}", alpha=0.7)
plt.title('UMAP Visualization of Market Regimes (K-means)')
plt.xlabel('UMAP Component 1')
plt.ylabel('UMAP Component 2')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('final/umap_kmeans_clusters.png', dpi=300, bbox_inches='tight')
plt.close()

# 5. Time series visualization of regimes (K-means)
# Sort by time
time_sorted_data = data_for_clustering.sort_values('Time')

# Plot regimes over time with price
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), gridspec_kw={'height_ratios': [3, 1]})

# Plot price on top subplot
if price_col:
    ax1.plot(time_sorted_data['Time'], time_sorted_data[price_col], color='black', alpha=0.7)
    ax1.set_ylabel('Price')
    ax1.set_title('Price and Market Regimes Over Time')
    ax1.grid(True, alpha=0.3)

# Plot regimes on bottom subplot
for i in range(k):
    regime_data = time_sorted_data[time_sorted_data['kmeans_cluster'] == i]
    ax2.scatter(regime_data['Time'], [i] * len(regime_data), c=[colors[i]], 
                label=f"Regime {i}: {kmeans_regimes[i]['name']}", alpha=0.7, s=10)

ax2.set_ylabel('Regime')
ax2.set_xlabel('Time')
ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax2.set_yticks(range(k))
ax2.set_yticklabels([f"{i}: {kmeans_regimes[i]['name']}" for i in range(k)])

# Format x-axis dates
fig.autofmt_xdate()
plt.tight_layout()
plt.savefig('final/regimes_over_time.png', dpi=300, bbox_inches='tight')
plt.close()

# 6. Volatility and regime visualization
if 'volatility_30s' in time_sorted_data.columns:
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(16, 12), gridspec_kw={'height_ratios': [2, 1, 1]})
    
    # Plot price
    if price_col:
        ax1.plot(time_sorted_data['Time'], time_sorted_data[price_col], color='black', alpha=0.7)
        ax1.set_ylabel('Price')
        ax1.set_title('Price, Volatility and Market Regimes Over Time')
        ax1.grid(True, alpha=0.3)
    
    # Plot volatility
    ax2.plot(time_sorted_data['Time'], time_sorted_data['volatility_30s'], color='red', alpha=0.7)
    ax2.set_ylabel('Volatility (30s)')
    ax2.grid(True, alpha=0.3)
    
    # Plot regimes
    for i in range(k):
        regime_data = time_sorted_data[time_sorted_data['kmeans_cluster'] == i]
        ax3.scatter(regime_data['Time'], [i] * len(regime_data), c=[colors[i]], 
                    label=f"Regime {i}", alpha=0.7, s=10)
    
    ax3.set_ylabel('Regime')
    ax3.set_xlabel('Time')
    ax3.set_yticks(range(k))
    ax3.set_yticklabels([f"{i}: {kmeans_regimes[i]['name']}" for i in range(k)])
    
    fig.autofmt_xdate()
    plt.tight_layout()
    plt.savefig('final/volatility_regimes_over_time.png', dpi=300, bbox_inches='tight')
    plt.close()

# ----- REGIME CHANGE ANALYSIS -----
print("\n----- REGIME CHANGE ANALYSIS -----")

def analyze_regime_transitions(time_series_data, cluster_column):
    """Analyze regime transitions and compute transition probabilities"""
    # Sort by time
    sorted_data = time_series_data.sort_values('Time')
    
    # Get cluster labels
    labels = sorted_data[cluster_column].values
    
    # Create transition matrix
    unique_labels = np.unique(labels)
    n_labels = len(unique_labels)
    
    # Initialize transition count matrix
    transition_counts = np.zeros((n_labels, n_labels))
    
    # Count transitions
    for i in range(len(labels) - 1):
        from_idx = np.where(unique_labels == labels[i])[0][0]
        to_idx = np.where(unique_labels == labels[i+1])[0][0]
        transition_counts[from_idx, to_idx] += 1
    
    # Convert to probabilities
    transition_probs = np.zeros_like(transition_counts, dtype=float)
    row_sums = transition_counts.sum(axis=1)
    
    for i in range(n_labels):
        if row_sums[i] > 0:
            transition_probs[i, :] = transition_counts[i, :] / row_sums[i]
    
    return transition_counts, transition_probs, unique_labels

# Analyze K-means regime transitions
kmeans_counts, kmeans_probs, kmeans_labels_unique = analyze_regime_transitions(
    data_for_clustering, 'kmeans_cluster'
)

# Create transition probability heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(kmeans_probs, annot=True, cmap='viridis', fmt='.2f',
           xticklabels=[f"{i}: {kmeans_regimes[i]['name']}" for i in kmeans_labels_unique],
           yticklabels=[f"{i}: {kmeans_regimes[i]['name']}" for i in kmeans_labels_unique])
plt.xlabel('To Regime')
plt.ylabel('From Regime')
plt.title('Regime Transition Probabilities')
plt.tight_layout()
plt.savefig('final/regime_transition_probs.png', dpi=300, bbox_inches='tight')
plt.close()

# Create transition count heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(kmeans_counts, annot=True, cmap='viridis', fmt='g',
           xticklabels=[f"{i}: {kmeans_regimes[i]['name']}" for i in kmeans_labels_unique],
           yticklabels=[f"{i}: {kmeans_regimes[i]['name']}" for i in kmeans_labels_unique])
plt.xlabel('To Regime')
plt.ylabel('From Regime')
plt.title('Regime Transition Counts')
plt.tight_layout()
plt.savefig('final/regime_transition_counts.png', dpi=300, bbox_inches='tight')
plt.close()

# Find most common regime transitions
transitions = []
for i in range(len(kmeans_labels_unique)):
    for j in range(len(kmeans_labels_unique)):
        if i != j:  # Exclude self-transitions
            transitions.append({
                'from_regime': f"Regime {kmeans_labels_unique[i]}: {kmeans_regimes[kmeans_labels_unique[i]]['name']}",
                'to_regime': f"Regime {kmeans_labels_unique[j]}: {kmeans_regimes[kmeans_labels_unique[j]]['name']}",
                'probability': kmeans_probs[i, j],
                'count': kmeans_counts[i, j]
            })

# Sort by probability
transitions_df = pd.DataFrame(transitions)
top_transitions = transitions_df.sort_values('probability', ascending=False).head(10)

print("\nTop Most Likely Regime Transitions:")
for i, row in top_transitions.iterrows():
    print(f"{row['from_regime']} → {row['to_regime']}: {row['probability']:.2f} (Count: {int(row['count'])})")

# Calculate regime duration statistics
def calculate_regime_durations(time_series_data, cluster_column):
    """Calculate how long each regime typically lasts"""
    # Sort by time
    sorted_data = time_series_data.sort_values('Time')
    
    # Convert time to seconds from start for easier calculation
    sorted_data['time_seconds'] = (sorted_data['Time'] - sorted_data['Time'].min()).dt.total_seconds()
    
    # Initialize variables
    regime_durations = {}
    current_regime = None
    start_time = None
    
    # Iterate through data
    for i, row in sorted_data.iterrows():
        regime = row[cluster_column]
        
        if current_regime is None:
            # First data point
            current_regime = regime
            start_time = row['time_seconds']
        elif regime != current_regime:
            # Regime changed
            duration = row['time_seconds'] - start_time
            
            if current_regime not in regime_durations:
                regime_durations[current_regime] = []
            
            regime_durations[current_regime].append(duration)
            
            # Reset for new regime
            current_regime = regime
            start_time = row['time_seconds']
    
    # Add the last regime
    if current_regime is not None:
        duration = sorted_data['time_seconds'].iloc[-1] - start_time
        
        if current_regime not in regime_durations:
            regime_durations[current_regime] = []
        
        regime_durations[current_regime].append(duration)
    
    # Calculate statistics
    regime_stats = {}
    for regime, durations in regime_durations.items():
        regime_stats[regime] = {
            'mean_duration': np.mean(durations),
            'median_duration': np.median(durations),
            'min_duration': np.min(durations),
            'max_duration': np.max(durations),
            'count': len(durations)
        }
    
    return regime_stats

# Calculate regime durations
regime_stats = calculate_regime_durations(data_for_clustering, 'kmeans_cluster')

# Print regime duration statistics
print("\nRegime Duration Statistics:")
for regime, stats in regime_stats.items():
    print(f"\nRegime {regime} - {kmeans_regimes[regime]['name']}:")
    print(f"  Mean Duration: {stats['mean_duration']:.2f} seconds")
    print(f"  Median Duration: {stats['median_duration']:.2f} seconds")
    print(f"  Min Duration: {stats['min_duration']:.2f} seconds")
    print(f"  Max Duration: {stats['max_duration']:.2f} seconds")
    print(f"  Number of Occurrences: {stats['count']}")

# Create a summary DataFrame
regime_summary = pd.DataFrame([
    {
        'Regime': f"{i}: {kmeans_regimes[i]['name']}",
        'Samples': kmeans_regimes[i]['count'],
        'Percentage': f"{kmeans_regimes[i]['percentage']:.2f}%",
        'Mean_Duration': f"{regime_stats[i]['mean_duration']:.2f}s",
        'Median_Duration': f"{regime_stats[i]['median_duration']:.2f}s",
        'Trend_Slope': f"{kmeans_regimes[i]['trend_slope']:.6f}",
        'RSI': f"{kmeans_regimes[i]['rsi']:.2f}",
        'Volatility': f"{kmeans_regimes[i]['volatility']:.6f}",
        'Spread_bps': f"{kmeans_regimes[i]['spread_bps']:.2f}",
        'Depth': f"{kmeans_regimes[i]['depth']:.2f}"
    }
    for i in kmeans_regimes
])

# Save summary to CSV
regime_summary.to_csv('final/regime_summary.csv', index=False)
print("\nRegime summary saved to 'final/regime_summary.csv'")

# Save the clustering results for later use
np.save('final/pca_model.npy', pca)
np.save('final/kmeans_model.npy', kmeans)
np.save('final/regime_mapping.npy', {k: v['name'] for k, v in kmeans_regimes.items()})
print("\nModels saved to 'final/' directory")

Using 31 features for clustering
After dropping NaN values: 1526373 rows
Applying PCA...
PCA reduced dimensions from 31 to 3
Explained variance ratio: [0.75425946 0.18850756 0.04732261]
Cumulative explained variance: 0.9900896343857355

----- K-MEANS CLUSTERING WITH K=3 -----

----- REGIME LABELING AND ANALYSIS -----

K-Means Regime Analysis:

Cluster 0 - Downward-Trending & Stable & Illiquid
  Samples: 443583 (29.06%)
  Trend: Downward-Trending (Slope=-0.005714, RSI=37.50)
  Volatility: Stable (0.000077)
  Liquidity: Illiquid (Spread=0.00 bps, Depth=-0.01)

Cluster 1 - Upward-Trending & Stable & Illiquid
  Samples: 448617 (29.39%)
  Trend: Upward-Trending (Slope=0.005686, RSI=63.40)
  Volatility: Stable (0.000073)
  Liquidity: Illiquid (Spread=0.03 bps, Depth=0.00)

Cluster 2 - Upward-Trending & Stable & Illiquid
  Samples: 634173 (41.55%)
  Trend: Upward-Trending (Slope=0.000132, RSI=50.36)
  Volatility: Stable (0.000074)
  Liquidity: Illiquid (Spread=-0.02 bps, Depth=0.00)

----- DI